<a href="https://colab.research.google.com/github/Rishikesh75/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
!pip install transformers accelerate sentence-transformers scikit-learn faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import requests
from bs4 import BeautifulSoup
import json

# Define your URLs by category
faq_links = {
    "payments": [
        "https://jupiter.money/payments/",
        "https://jupiter.money/bills-recharges/",
        "https://jupiter.money/pay-via-upi/",
        "https://jupiter.money/magic-spends/"
    ],
    "credit_card": [
        "https://jupiter.money/edge-plus-upi-rupay-credit-card/",
        "https://jupiter.money/edge-csb-rupay-credit-card/",
        "https://jupiter.money/edge-visa-credit-card/"
    ],
    "rewards": [
        "https://jupiter.money/rewards/"
    ],
    "loans": [
        "https://jupiter.money/loan/",
        "https://jupiter.money/loan-against-mutual-funds/"
    ],
    "investments": [
        "https://jupiter.money/investments/",
        "https://jupiter.money/mutual-funds/",
        "https://jupiter.money/digi-gold/",
        "https://jupiter.money/flexi-fd/",
        "https://jupiter.money/recurring-deposits/",
        "https://jupiter.money/magic-spends/"
    ],
    "money": [
        "https://jupiter.money/money/"
    ]
}

# Extraction function
def extract_faqs_from_url(url, category):
    faqs = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        faq_items = soup.find_all("div", class_="faq-item")

        for item in faq_items:
            question_tag = item.find("div", class_="faq-header").find("span")
            question = question_tag.get_text(strip=True) if question_tag else "No question found"

            answer_tag = item.find("div", class_="faq-answer")
            if answer_tag:
                paragraphs = [p.get_text(strip=True) for p in answer_tag.find_all("p")]
                list_items = [li.get_text(strip=True) for li in answer_tag.find_all("li")]
                answer = "\n".join(paragraphs + list_items)
            else:
                answer = "No answer found"

            faqs.append({
                "question": question,
                "answer": answer,
                "category": category,
                "source_url": url
            })

    except Exception as e:
        print(f"Error while processing {url}: {e}")

    return faqs

# Final dataset
full_dataset = []

# Go through each category and URL
for category, urls in faq_links.items():
    for url in urls:
        data = extract_faqs_from_url(url, category)
        full_dataset.extend(data)

# Preview some entries
for i, item in enumerate(full_dataset[:5], 1):
    print(f"Q{i}: {item['question']}")
    print(f"A{i}: {item['answer']}")
    print(f"Category: {item['category']}, URL: {item['source_url']}\n")

# Save as JSON
with open("/content/drive/MyDrive/JupyterCompany/jupiter_faq_dataset.json", "w", encoding="utf-8") as f:
    json.dump(full_dataset, f, indent=2, ensure_ascii=False)

print(f"\n✅ Total FAQs extracted: {len(full_dataset)}")



Q1: What payment methods does Jupiter support?
A1: You can make payments via UPI, your linked bank account, and debit cards. You can also link any bank’s RuPay
            Credit Card to Jupiter UPI and make payments.
Category: payments, URL: https://jupiter.money/payments/

Q2: Is it safe to make payments
            through Jupiter?
A2: Yes. Jupiter uses industry-standard encryption and is compliant with RBI regulations to ensure secure and
            private transactions.
Category: payments, URL: https://jupiter.money/payments/

Q3: Can I pay my bills using Jupiter?
A3: Yes, you can pay utility bills like electricity, gas, DTH, water, and mobile recharges directly from the
            app.
Category: payments, URL: https://jupiter.money/payments/

Q4: Can I link multiple bank accounts to Jupiter?
A4: Yes, you can link more than one bank account and switch between them when making UPI payments.
Category: payments, URL: https://jupiter.money/payments/

Q5: Can I track my payments on J

In [5]:
# prompt: Normalize and deduplicate similar questions.
# Clean any HTML or formatting noise.
# Categorize the content into topics such as KYC, rewards, payments, limits, etc. and the File is here
# /content/drive/MyDrive/JupyterCompany/jupiter_faq_dataset.json


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer, util
import numpy as np
import json
import re

# Load the dataset
file_path = "/content/drive/MyDrive/JupyterCompany/jupiter_faq_dataset.json"
try:
    with open(file_path, "r", encoding="utf-8") as f:
        full_dataset = json.load(f)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    # If file is not found, use the extraction logic from the previous step
    # This part assumes the extraction code provided in the prompt is still in memory/runnable
    print("Attempting to extract data instead...")
    full_dataset = []
    # Re-run the extraction (copy the extraction logic from the prompt)
    for category, urls in faq_links.items():
        for url in urls:
            data = extract_faqs_from_url(url, category)
            full_dataset.extend(data)
    # Save the extracted data if it was not found
    if full_dataset:
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(full_dataset, f, indent=2, ensure_ascii=False)
        print(f"\n✅ Data extracted and saved to {file_path}")
    else:
        print("❌ Failed to extract any data. Cannot proceed with normalization and deduplication.")
        exit() # Exit if no data is available

# --- Normalization and Cleaning ---

def clean_text(text):
    # Remove HTML tags
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    # Remove extra whitespace and newline characters
    text = re.sub('\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()
    return text

# Apply cleaning to questions and answers
cleaned_dataset = []
for item in full_dataset:
    cleaned_dataset.append({
        "question": clean_text(item["question"]),
        "answer": clean_text(item["answer"]),
        "category": item["category"],
        "source_url": item["source_url"]
    })

# --- Deduplication (using Sentence Embeddings and Clustering) ---

# Extract cleaned questions
questions = [item["question"] for item in cleaned_dataset]

# Use a pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# Compute embeddings for each question
print("Computing sentence embeddings...")
sentence_embeddings = model.encode(questions, convert_to_numpy=True, show_progress_bar=True)

# Define a similarity threshold (cosine similarity)
# Adjust this threshold based on how strict you want the deduplication to be
similarity_threshold = 0.90 # Closer to 1 means very similar questions are grouped

# Use Agglomerative Clustering to group similar questions
# We use affinity='cosine' and linkage='average' for sentence embeddings
print(f"Clustering questions with similarity threshold > {similarity_threshold}...")
clustering_model = AgglomerativeClustering(n_clusters=None, metric='cosine', linkage='average', distance_threshold=1 - similarity_threshold)
clustering_model.fit(sentence_embeddings)

# Get cluster labels for each question
labels = clustering_model.labels_

# Create a dictionary to store unique questions per cluster
clustered_faqs = {}
for i, label in enumerate(labels):
    if label not in clustered_faqs:
        clustered_faqs[label] = []
    clustered_faqs[label].append(cleaned_dataset[i])

# Select one representative question from each cluster
# For simplicity, we'll take the first question in each cluster as the representative
deduplicated_dataset = []
for cluster_label, faqs_in_cluster in clustered_faqs.items():
    # You could add logic here to select the "best" representation
    # e.g., based on length, source URL priority, etc.
    representative_faq = faqs_in_cluster[0]
    deduplicated_dataset.append(representative_faq)

# --- Results ---
print(f"\nOriginal number of FAQs: {len(full_dataset)}")
print(f"Cleaned number of FAQs: {len(cleaned_dataset)}")
print(f"Deduplicated number of unique question clusters: {len(deduplicated_dataset)}")

# Preview some deduplicated entries
print("\n--- Preview of Deduplicated FAQs ---")
for i, item in enumerate(deduplicated_dataset[:10], 1):
    print(f"Q{i} (Category: {item['category']}): {item['question']}")
    print(f"A{i}: {item['answer'][:150]}...") # Print truncated answer
    print("-" * 20)

# You can optionally save the deduplicated dataset
with open("/content/drive/MyDrive/JupyterCompany/jupiter_faq_dataset_deduplicated.json", "w", encoding="utf-8") as f:
    json.dump(deduplicated_dataset, f, indent=2, ensure_ascii=False)
print("\nDeduplicated dataset saved to /content/drive/MyDrive/JupyterCompany/jupiter_faq_dataset_deduplicated.json")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computing sentence embeddings...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Clustering questions with similarity threshold > 0.9...

Original number of FAQs: 103
Cleaned number of FAQs: 103
Deduplicated number of unique question clusters: 90

--- Preview of Deduplicated FAQs ---
Q1 (Category: payments): what payment methods does jupiter support?
A1: you can make payments via upi, your linked bank account, and debit cards. you can also link any bank’s rupay credit card to jupiter upi and make payme...
--------------------
Q2 (Category: payments): is it safe to make payments through jupiter?
A2: yes. jupiter uses industry-standard encryption and is compliant with rbi regulations to ensure secure and private transactions....
--------------------
Q3 (Category: payments): can i pay my bills using jupiter?
A3: yes, you can pay utility bills like electricity, gas, dth, water, and mobile recharges directly from the app....
--------------------
Q4 (Category: payments): can i link multiple bank accounts to jupiter?
A4: yes, you can link more than one bank account and sw

In [15]:
import json
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import List
import numpy as np

# Step 1: Load the JSON FAQ data
with open("/content/drive/MyDrive/JupyterCompany/jupiter_faq_dataset_deduplicated.json", "r", encoding="utf-8") as f:
    faq_data = json.load(f)

# Step 2: Prepare corpus (we use question + answer as context for better results)
corpus = [f"Q: {item['question']} A: {item['answer']}" for item in faq_data]
questions_only = [item["question"] for item in faq_data]

# Step 3: Embed the corpus using SentenceTransformers
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Small and fast
corpus_embeddings = embedding_model.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

# Step 4: Create FAISS index
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings.cpu().detach().numpy())  # FAISS requires numpy arrays

# Step 5: Load a lightweight Hugging Face LLM for generation
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Step 6.1: Keep conversation history
conversation_history: List[str] = []

def get_answer(user_question: str, top_k: int = 2):
    global conversation_history

    # Embed current user question
    question_embedding = embedding_model.encode(user_question, convert_to_tensor=True)
    D, I = index.search(question_embedding.cpu().detach().numpy().reshape(1, -1), top_k)
    retrieved_contexts = [corpus[i] for i in I[0]]

    # Build conversational context
    context = "Previous Conversation:\n"
    for turn in conversation_history[-4:]:  # last 4 exchanges
        context += turn + "\n"

    faq_context = "\n".join(retrieved_contexts)
    prompt = f"""You are a helpful assistant trained on the following FAQs:
      {faq_context}

      Only use this information to answer the question. If the answer is not found in these FAQs, reply:
        "I don’t have that information. I have only been trained on Jupiter's FAQs related to payments, UPI, credit cards, and account features."

      Example:
        User Question: What is the capital of France?
        Answer: I don’t have that information. I have only been trained on Jupiter's FAQs related to payments, UPI, credit cards, and account features.

      Conversation history:
        {context}

      User Question: {user_question}
      Answer:"""

    # Generate with model
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Add to history
    conversation_history.append(f"User: {user_question}")
    conversation_history.append(f"Assistant: {answer}")

    return answer



Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# Initialize history list
qa_history = []

# Input widget for user question
input_box = widgets.Text(
    value='',
    placeholder='Type your question here',
    description='Question:',
    layout=widgets.Layout(width='80%')
)

# Button to submit the question
submit_button = widgets.Button(
    description='Ask',
    button_style='success',
    tooltip='Click to get an answer',
    icon='question'
)

# Output area for current answer and history
current_output = widgets.Output()
history_output = widgets.Output()

# Function to update the history display
def update_history():
    with history_output:
        clear_output()
        for idx, (q, a) in enumerate(qa_history, 1):
            display(Markdown(f"## Q{idx}: {q}\n ###### A{idx}: {a}\n---"))

# On submit function
def on_submit(_):
    with current_output:
        clear_output()
        user_q = input_box.value.strip()
        if user_q.lower() == "exit":
            print("Exited.")
            return
        if not user_q:
            print("Please enter a valid question.")
            return

        answer = get_answer(user_q)
        qa_history.append((user_q, answer))
        print(f"Answer: {answer}")

        input_box.value = ""  # Clear input box
        update_history()

# Link button click to function
submit_button.on_click(on_submit)

# Display layout
display(widgets.VBox([
    input_box,
    submit_button,
    widgets.Label("Latest Answer:"),
    current_output,
    widgets.Label("Question & Answer History:"),
    history_output
]))
